In [6]:
import napari 
from skimage import io

original_tiff_image_series = "C:/Users/micha/Desktop/Master_Project/tiff_file_converted.tiff"
original_image_series = io.imread(original_tiff_image_series)

viewer = napari.view_image(original_image_series, colormap='gray', name ='Original Image Series')

In [13]:
import napari
from skimage import io
import cv2
import numpy as np
from skimage.segmentation import watershed

def watershed_segmentation(image):
    # Convert to grayscale if the image is RGB
    if image.ndim == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Otsu's Thresholding
    _, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    binary_image = np.uint8(binary_image)

    # Apply Morphological Operations (optional)
    kernel = np.ones((5, 5), np.uint8)
    binary_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)

    # Apply Distance Transform and Threshold
    distance_transform = cv2.distanceTransform(binary_image, cv2.DIST_L2, 3)
    _, sure_fg = cv2.threshold(distance_transform, 0.7 * distance_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)

    # Finding Sure Background
    sure_bg = cv2.dilate(binary_image, kernel, iterations=3)

    # Finding Unknown Region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Label Markers for Watershed Segmentation
    _, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0

    # Apply Watershed Algorithm
    labels = watershed(-distance_transform, markers, mask=binary_image)

    # Overlay the segmentation on the original image
    overlay = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    overlay[labels > 1] = [0, 0, 255]  # Color the segmented regions in red

    return overlay

# Read the image stack
original_tiff_image_series = "C:/Users/micha/Desktop/Master_Project/tiff_file_converted.tiff"
image_stack = io.imread(original_tiff_image_series)

# Process and visualize images in batches
batch_size = 5
for i in range(0, len(image_stack), batch_size):
    batch = image_stack[i:i + batch_size]
    segmented_batch = [watershed_segmentation(slice_) for slice_ in batch]

    # Visualize the segmented batch in Napari
    viewer = napari.view_image(np.array(segmented_batch), colormap='gray', name='Segmented Image Stack')


MemoryError: Unable to allocate 4.04 GiB for an array with shape (392, 2160, 2560) and data type uint16